<a href="https://colab.research.google.com/github/mfrdixon/alpha-RNN/blob/master/KerasLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 14 23:15:31 2020

@author: macbookpro
"""
 

# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
from numpy import random
import os# Generate switching data set
import random


# Imports for alpha_rnns 
from IPython import display
import tensorflow.compat.v1 as tf   
tf.disable_v2_behavior()
# Imports for stats
from keras.layers import Dense, Input
from keras.models import Model
import numpy as np
from keras.layers import Layer
from keras import backend as K
from keras import optimizers
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU, SimpleRNN
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.regularizers import l1,l2
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from keras import layers
#from alphaRNN import *

# To make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot figures
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)
    
def generate_vol_sample(length, sigma_0, n_steps, step_size, p, eps=0.01, shift=0):
    sigma = np.array([0]*length, dtype='float64')
    sigma[0]=sigma_0
    mu = np.array([0]*length, dtype='float64')
    phi = np.array([0]*length*p, dtype='float64').reshape(length,p)
    #phi2 = np.array([0]*length, dtype='float64')
    step_length=100 #np.int(np.floor(np.float(length)/(2.0*n_steps)))
    
    for i in range(2*n_steps):
      #mu[i*step_length:((i*step_length)+1)]=step_size #*(-1)**i
      mu[i*step_length:((i+1)*step_length)]= step_size*(-1)**i
      if i%2==0:  
        phi[i*step_length:((i+1)*step_length),:]= 0.02
        #phi2[i*step_length:((i+1)*step_length)]=1.0
      else:
        phi[i*step_length:((i+1)*step_length),:]=0.01
        #phi2[i*step_length:((i+1)*step_length)]=0.5
    for i in range(p, length):
        sigma[i]= mu[i-1] + np.random.normal(0,eps)
        for j in range(p):
          sigma[i]+=phi[i-1,j]*sigma[i-j]  
        
    return (sigma+shift)

p = 30 # the number of lags (in both the data and the models)
vols=generate_vol_sample(2000, 0.25, 15, 0.1, p, 1e-4, 0.13)[p:]

df = pd.DataFrame(vols, columns=['vol'])

use_features = ['vol'] 
target = 'vol'
n_steps = 10 # number of lags to include in the model

train_weight = 0.8
split = int(len(df)*train_weight)

df_train = df[use_features].iloc[:split]
print(df_train)
df_test = df[use_features].iloc[split:]

def get_lagged_features(value, n_steps):
    lag_list = []
    for lag in range(n_steps, 0, -1):
        lag_list.append(value.shift(lag))
    return pd.concat(lag_list, axis=1)

x_train_list = []
for use_feature in use_features:
    x_train_reg = get_lagged_features(df_train, n_steps).dropna()
    x_train_list.append(x_train_reg)
x_train_reg = pd.concat(x_train_list, axis=1)

col_ords = []
for i in range(n_steps):
    for j in range(len(use_features)):
        col_ords.append(i + j * n_steps)

x_train_reg = x_train_reg.iloc[:, col_ords]
y_train_reg = df_train.loc[x_train_reg.index, [target]].values
x_train_reg = np.reshape(x_train_reg.values, (x_train_reg.shape[0], np.int(x_train_reg.shape[1] / len(use_features)), len(use_features)))
y_train_reg = np.reshape(y_train_reg, (y_train_reg.shape[0], 1, 1))

x_test_list = []
for use_feature in use_features:
    x_test_reg = get_lagged_features(df_test, n_steps).dropna()
    x_test_list.append(x_test_reg)
x_test_reg = pd.concat(x_test_list, axis=1)

x_test_reg = x_test_reg.iloc[:, col_ords]
y_test_reg = df_test.loc[x_test_reg.index, [target]].values
x_test_reg = np.reshape(x_test_reg.values, (x_test_reg.shape[0], np.int(x_test_reg.shape[1]/len(use_features)), len(use_features)))

y_test_reg = np.reshape(y_test_reg, (y_test_reg.shape[0], 1, 1))

train_batch_size = y_train_reg.shape[0]
test_batch_size = y_test_reg.shape[0]    

class SimpleAlphaRNN(Layer):

    def __init__(self, input_dimensions, hidden_size=5, output_dim=1, dtype=tf.float64, **kwargs):
        self.input_dimensions = input_dimensions
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.Wh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wh')
        # Weights for hidden vectors of shape (hidden_size, hidden_size)
        self.Uh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Uh')
        # Biases for hidden vectors of shape (hidden_size,)
        self.bh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='bh')        
        # Define the input layer placeholder
        self.input_layer = tf.placeholder(dtype=tf.float64, shape=(None, None, input_dimensions), name='input')
        self.alpha = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='alpha')        
        #tf.placeholder(dtype=tf.float64, shape=(None, 1), name='alpha')
        # Put the time-dimension upfront for the scan operator
        self.x_t = tf.transpose(self.input_layer, [1, 0, 2], name='x_t')        
        # A little hack (to obtain the same shape as the input matrix) to define the initial hidden state h_0
        self.h_0 = tf.multiply(self.x_t[0, :, :], tf.zeros(dtype=tf.float64, shape=(input_dimensions, hidden_size)),name='h_0')        
        # Perform the scan operator
        self.h_t_transposed = tf.scan(self.call, self.x_t, initializer=self.h_0, name='h_t_transposed')        
        # Transpose the result back
        self.h_t = tf.transpose(self.h_t_transposed, [1, 0, 2], name='h_t')
        self.W_output = tf.Variable(tf.truncated_normal(dtype=tf.float64, shape=(hidden_size, 1), mean=0, stddev=0.01))
        self.b_output = tf.Variable(tf.truncated_normal(dtype=tf.float64, shape=(1,), mean=0, stddev=0.01))
        self._output = tf.multiply(self.h_t, self.W_output) + self.b_output 

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.Uh = self.add_weight(name='Uh',
                                      shape=(self.hidden_size, self.hidden_size),
                                      initializer='uniform',
                                      trainable=True)

        self.Wh = self.add_weight(name='Wh',
                                           shape=(self.input_dimensions,self.hidden_size),
                                           initializer='uniform',
                                           trainable=True)
        
        self.W_output = self.add_weight(name='W_output',
                                           shape=(hidden_size,1),
                                           initializer='uniform',
                                           trainable=True)
        super(SimpleAlphaRNN, self).build(input_shape)  # Be sure to call this at the end
        
    def call(self, h_tml, x_t):
        h_proposal = tf.tanh(tf.matmul(x_t, self.Wh) + tf.matmul(h_tml, self.Uh) + self.bh)
        return h_proposal
        
    #def call2(self, h_tml, x_t):
        # Update hidden state h_t
    #    h_proposal = np.math.tanh(K.dot(x_t, self.Wh) + K.dot(h_tml, self.Uh) + self.bh)
        # Update the (smoothed) hidden state with exponential smoothing
    #    h_t = K.dot(1 - np.math.tanh(self.alpha), h_tm1) + K.dot(np.math.tanh(self.alpha), h_proposal)
    #    return h_t 

    #def call(self, x):
        #return K.dot(x, self.kernel)
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

input_dimension = 1
hidden_units = 5
#model = Sequential()
model_in = Input(shape=(input_dimension, hidden_units))  
#model_out = 
#model.
#model.add(SimpleAlphaRNN(input_dimension,hidden_units,1))
#model_layer = layers.Layer(SimpleAlphaRNN(input_dimension,hidden_units))(input)
#model.add(model_layer)
#model.add(Dense(1))
model_out = SimpleAlphaRNN(input_dimension,hidden_units)  #(model_in)
#layers.Dense(Input, activation='sigmoid')(Input)
model =  Model(inputs=[model_in], outputs=[model_out])
adamOpt = optimizers.Adam(lr=1e-4)
model.compile(optimizer=adamOpt, loss='mean_squared_error')
hist = model.fit(x_train_reg,y_train_reg,verbose=2, epochs=2000, batch_size=100)

           vol
0     0.231760
1     0.234163
2     0.236118
3     0.238111
4     0.240346
5     0.242620
6     0.244921
7     0.246864
8     0.249513
9     0.251972
10    0.254264
11    0.256771
12    0.259587
13    0.261881
14    0.264645
15    0.267366
16    0.270018
17    0.272699
18    0.275689
19    0.278759
20    0.281803
21    0.284439
22    0.287653
23    0.290854
24    0.294061
25    0.297234
26    0.300527
27    0.304021
28    0.307590
29    0.311187
...        ...
1546 -0.011436
1547 -0.011533
1548 -0.011484
1549 -0.011600
1550 -0.011823
1551 -0.011848
1552 -0.011636
1553 -0.011656
1554 -0.011709
1555 -0.011783
1556 -0.011949
1557 -0.011762
1558 -0.011780
1559 -0.011837
1560 -0.011790
1561 -0.011746
1562 -0.012055
1563 -0.012101
1564 -0.012253
1565 -0.011793
1566 -0.011939
1567 -0.012213
1568 -0.012138
1569 -0.012121
1570 -0.012194
1571  0.149624
1572  0.153028
1573  0.156368
1574  0.159760
1575  0.163050

[1576 rows x 1 columns]


ValueError: Output tensors to a Model must be the output of a Keras `Layer` (thus holding past layer metadata). Found: <__main__.SimpleAlphaRNN object at 0x103272be0>